<a href="https://colab.research.google.com/github/gharbiaziz10/AirCraft-2004/blob/main/AirCraft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Year, Month, DayofMonth and DayOfWeek:** dates of the flight

**DepTime and ArrTime:** real times at which take-off and landing took place

**CRSDepTime, CRSArrTime :** Planned Departure and Arrival Time

**UniqueCarrier :** Airline identifier

**FlightNum :** Flight Number

**TailNum :** Plane identifier

**ActualElapsedTime ;** Duration of the flight in minutes

**CRSElapsedTime :** Planned duration of the flight in minutes

**AirTime :** Duration of the flight in the air

**ArrDelay :** Total Delay on Arrival in minute

**DepDelay :** Departure delay (our target)

**Origin :** Starting Airport Code

**Dest :** Destination

**Distance :**  Distance between origin and destination

**TaxiOut :** time spent taxiing before departure

**Taxiln :** time spent taxiing after landing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [ ]:
df=pd.read_csv('2004.csv')

**EDA**

In [ ]:
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2004,1,12,1,623.0,630,901.0,915,UA,462,...,7,11,0,NaN,0,0,0,0,0,0
1,2004,1,13,2,621.0,630,911.0,915,UA,462,...,16,16,0,NaN,0,0,0,0,0,0
2,2004,1,14,3,633.0,630,920.0,915,UA,462,...,4,15,0,NaN,0,0,0,0,0,0
3,2004,1,15,4,627.0,630,859.0,915,UA,462,...,4,10,0,NaN,0,0,0,0,0,0
4,2004,1,16,5,635.0,630,918.0,915,UA,462,...,3,13,0,NaN,0,0,0,0,0,0


In [ ]:
df.shape

(299, 29)

In [ ]:
df.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,299.0,299.0,299.000000,299.000000,289.000000,299.000000,288.000000,299.000000,299.000000,288.000000,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,299.000000,299.0,299.000000
mean,2004.0,1.0,16.474916,4.093645,1506.169550,1510.682274,1682.375000,1727.036789,465.973244,157.079861,...,1027.779264,5.595318,15.066890,0.033445,0.003344,3.120401,0.0,2.772575,0.0,5.183946
std,0.0,0.0,8.868647,1.938118,544.489276,528.046224,551.470386,486.355404,2.029800,67.562777,...,508.564322,3.806579,9.209468,0.180096,0.057831,12.738683,0.0,7.970275,0.0,17.185416
min,2004.0,1.0,1.000000,1.000000,10.000000,630.000000,2.000000,526.000000,462.000000,52.000000,...,236.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
25%,2004.0,1.0,9.000000,2.000000,947.000000,1330.000000,1229.000000,1230.000000,465.000000,95.000000,...,533.000000,3.000000,10.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
50%,2004.0,1.0,17.000000,4.000000,1635.000000,1640.000000,1906.000000,1915.000000,466.000000,146.500000,...,992.000000,5.000000,12.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
75%,2004.0,1.0,24.000000,6.000000,2005.000000,2000.000000,2109.250000,2114.000000,468.000000,225.500000,...,1671.000000,6.000000,17.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
max,2004.0,1.0,31.000000,7.000000,2331.000000,2330.000000,2342.000000,2332.000000,470.000000,312.000000,...,1835.000000,25.000000,63.000000,1.000000,1.000000,135.000000,0.0,50.000000,0.0,158.000000


In [ ]:
# la ligne count montre des cardinux différents :
#-- > traitement des données manquantes
# les données sont très hétérogènes et d'ordre de grandeur très variables
# -- > Standardisation des données
# si on va utiliser des algorithmes qui opèrent avec les distances
# -- > il ya des features non significatifs comme nombre de mois, ordre du jour
# -- > nettoyage de données'''

In [ ]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [ ]:
df.fillna(method='ffill',inplace=True)

<ipython-input-15-ae2c85fc64fc>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill',inplace=True)


In [ ]:
df.describe()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,ActualElapsedTime,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,299.0,299.0,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,...,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.0,299.000000,299.0,299.000000
mean,2004.0,1.0,16.474916,4.093645,1508.612040,1510.682274,1677.775920,1727.036789,465.973244,159.193980,...,1027.779264,5.595318,15.066890,0.033445,0.003344,3.120401,0.0,2.772575,0.0,5.183946
std,0.0,0.0,8.868647,1.938118,543.617312,528.046224,557.198096,486.355404,2.029800,68.096969,...,508.564322,3.806579,9.209468,0.180096,0.057831,12.738683,0.0,7.970275,0.0,17.185416
min,2004.0,1.0,1.000000,1.000000,10.000000,630.000000,2.000000,526.000000,462.000000,52.000000,...,236.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
25%,2004.0,1.0,9.000000,2.000000,947.000000,1330.000000,1228.500000,1230.000000,465.000000,97.000000,...,533.000000,3.000000,10.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
50%,2004.0,1.0,17.000000,4.000000,1634.000000,1640.000000,1905.000000,1915.000000,466.000000,149.000000,...,992.000000,5.000000,12.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
75%,2004.0,1.0,24.000000,6.000000,2005.000000,2000.000000,2109.500000,2114.000000,468.000000,227.500000,...,1671.000000,6.000000,17.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
max,2004.0,1.0,31.000000,7.000000,2331.000000,2330.000000,2342.000000,2332.000000,470.000000,312.000000,...,1835.000000,25.000000,63.000000,1.000000,1.000000,135.000000,0.0,50.000000,0.0,158.000000


**Analyse des vols annulés**

In [ ]:
#Spécifier les features pour l'analyse et le target
X=df[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
'AirTime', 'Distance' ]] #features
y=df ['Cancelled']

In [ ]:
# Subdiviser les données
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(X,y,shuffle=True,test_size=0.3,random_state=7)

In [ ]:
# Standardiser les données
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

**KNN**

In [ ]:
#importer le modèle
from sklearn. neighbors import KNeighborsClassifier
# Explorer le modèle
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train,y_train)
y_pred=knn.predict(x_test)
# Generer le rapport de classification
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        87
           1       1.00      0.33      0.50         3

    accuracy                           0.98        90
   macro avg       0.99      0.67      0.74        90
weighted avg       0.98      0.98      0.97        90



In [ ]:
# -- > KNN montre de meilleures affinités pour les vols non annulés
# -- > KNN montre des difficultés majeures à détecter les vols annulés
#puisque l'indice de sensibilité: recall s'avère très faible (29%)
#on peut juger les performances de classification de KNN de faibles

**Decision Tree**

In [ ]:
#Importer le modèle
from sklearn. tree import DecisionTreeClassifier
#Exploiter le modèle
dt=DecisionTreeClassifier(criterion='gini',max_depth=35)
dt.fit(x_train,y_train)
y_pred_dt=dt.predict(x_test)
#Génération du rapport de classification
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_dt))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        87
           1       0.33      0.33      0.33         3

    accuracy                           0.96        90
   macro avg       0.66      0.66      0.66        90
weighted avg       0.96      0.96      0.96        90



In [ ]:
#le modèle dt se révèle plus sensible que KNN pour la detection des
#vols annulés. Toutefois sa précision se détériore significativement

**Random Forest**

In [ ]:
#importer le modèle
from sklearn. ensemble import RandomForestClassifier
#Explorer le modèle
rf=RandomForestClassifier(n_estimators=150,criterion='gini',random_state=7)
rf.fit(x_train,y_train)
y_pred_rf=rf.predict(x_test)
#Génération du rapport de classification
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        87
           1       0.33      0.33      0.33         3

    accuracy                           0.96        90
   macro avg       0.66      0.66      0.66        90
weighted avg       0.96      0.96      0.96        90



**Deep Learning**

In [ ]:
# 4. Création d'une architecture Deep Learning
#Etape 1: Déclarer que la nature des données de la colonnes 'Cancelled'
#---> catégorique
#1. importer la méthode
from tensorflow.keras.utils import to_categorical
#2. exploiter la méthode
y_train_dl=to_categorical(y_train)
y_test_dl=to_categorical(y_test)
#Etape 2: Créer le réseau DL
#Etape1: faire les importations nécéssaire
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#Etape 2: Créer le réseau DL
model = Sequential()
model.add(Dense(64,activation='relu',input_dim=x_train.shape[1]))
model.add(Dense(32,activation='relu'))
model.add(Dense(2,activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
#Etape 3: Compiler le modèle
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
# Apprentissage du modèle
model.fit(x_train,y_train_dl,epochs=3,batch_size=32,verbose=1)

Epoch 1/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8052 - loss: 0.5854
Epoch 2/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9423 - loss: 0.3878 
Epoch 3/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9520 - loss: 0.2987 


In [ ]:
y_pred_dl=model.predict(x_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [ ]:
y_pred_dl

array([[0.94500476, 0.05499519],
       [0.7469058 , 0.25309414],
       [0.9569006 , 0.04309934],
       [0.8272527 , 0.17274727],
       [0.89310974, 0.10689014],
       [0.81551   , 0.18449004],
       [0.8787118 , 0.12128825],
       [0.93436533, 0.06563458],
       [0.8976476 , 0.10235231],
       [0.84684587, 0.1531541 ],
       [0.69345146, 0.30654845],
       [0.96865517, 0.03134485],
       [0.7298422 , 0.27015775],
       [0.80814534, 0.19185464],
       [0.66330105, 0.33669898],
       [0.9494165 , 0.05058332],
       [0.9295112 , 0.07048893],
       [0.789286  , 0.21071395],
       [0.84251064, 0.15748937],
       [0.8795294 , 0.12047067],
       [0.82978255, 0.17021745],
       [0.8216382 , 0.17836185],
       [0.8106204 , 0.18937963],
       [0.8376831 , 0.1623169 ],
       [0.90622276, 0.09377719],
       [0.6617089 , 0.3382911 ],
       [0.7585723 , 0.24142775],
       [0.7600864 , 0.23991358],
       [0.87177676, 0.12822326],
       [0.93747103, 0.06252898],
       [0.

In [ ]:
# Extraire le classe finale
# Cancelled ---> 1
# Cancelled ---> 0
y_pred_final=np.argmax(y_pred_dl,axis=1)
print(classification_report(y_test,y_pred_final))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98        87
           1       0.00      0.00      0.00         3

    accuracy                           0.97        90
   macro avg       0.48      0.50      0.49        90
weighted avg       0.93      0.97      0.95        90



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Tableau De Bord

In [ ]:
import ipywidgets as widgets
from IPython.display import display

In [ ]:
df.columns

Index(['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime',
       'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay',
       'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'],
      dtype='object')

In [ ]:
# Eviter les donnees Manquantes
df['DepDelay']=df['DepDelay'].fillna(0)
df['ArrDelay']=df['ArrDelay'].fillna(0)
# Ajouter une colonne  de donnees 'Delay Category'
df['Delay Category'] = df['DepDelay'].apply(lambda x: 'On Time' if x <= 0 else 'Delayed')
# Ajouter une colonne qui permet d'afficher les heures
df['DepTimeHour'] = df['DepTime']//100

In [ ]:
#Créer un widget qui permet de sélectionner lz compagnie aérienne
carrier_dropdown=widgets.Dropdown(options=df['UniqueCarrier'].unique(),description='compagnie :')
display(carrier_dropdown)

Dropdown(description='compagnie :', options=('UA',), value='UA')

In [ ]:
#Créer une fonction qui permet de construire les graphes du tableau de bord
def dashboard(carrier):
  #filtrer les données selon le carrier sélectionné dans dropdown
  filtered_df=df[df['UniqueCarrier']== carrier]
  #Créer le tableau de bord
  fig,axs=plt.subplots(2,2,figsize=(15,10))

  #premier graphe
  axs [0,0].set_title(f'distribution des retard de dépaet -{carrier}')
  axs[0,0].set_xlabel('Retard en minutes')
  axs[0,0].set_ylabel('Nombre de vols')
  sns.histplot(filtered_df['DepDelay'],bins=30,kde=True,ax=axs[0,0])

  #deuxième grapge
  axs[0,1].set_title('Retard moyen par compagnie aérienne')
  axs [0,1].set_xlabel('Retard en minutes' )
  axs[0,1].set_ylabel('Compagnie aérienne')
  mean_delays_by_carrier=df.groupby('UniqueCarrier')['DepDelay'].mean().sort_values()
  mean_delays_by_carrier.plot(kind='bar',color='blue',ax=axs[0,1])

  #troisième graphique
  axs[1,0].set_title(f"Retard en fonction de lheure - {carrier}")
  axs [1,0].set_xlabel ('Heure de départ' )
  axs[1,0].set_ylabel('Retard de départ')
  sns.boxplot(x='DepTimeHour',y='DepDelay',data=filtered_df,ax=axs[1,0])

  #Quatrième graphique
  axs[1,1].set_title(f'Repartition des vols à 1\'heure vs les retards - {carrier}')
  delay_counts=filtered_df['Delay Category' ].value_counts()
  axs[1,1].pie(delay_counts,labels=delay_counts.index,startangle=90)

  #Ajuster le dashbord selon la mise en page
  plt.tight_layout()

  #Affichage graphique
  plt.show()


In [ ]:
# Etablir le lien entre le widjet et la fonction
widgets.interact(dashboard,carrier=carrier_dropdown)

interactive(children=(Dropdown(description='compagnie :', options=('UA',), value='UA'), Output()), _dom_classe…

<function __main__.dashboard(carrier)>